# Projeto ICD - Regressão
## Regressão entre valor da aposta com base nos demais atributos

O objetivo é, em jogadas onde os jogadores aumentam as apostas, tentar prever qual o valor que será aumentado, a partir dos dados presentes na mesa.
Aplicaremos os modelos kNN e regressão linear com a variável prevista sendo o valor da aposta do jogador naquele turno, definida em "Raise". Verificaremos os valores de variância, R2, erro médio absoluto e erro quadrado médio para avaliar seus desempenhos na regressão. Por fim, compararemos os métodos com base nesses valores calculados. 

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import explained_variance_score, r2_score, mean_squared_error, median_absolute_error
from sklearn.feature_selection import RFE

### Carregando dados tratados:

In [2]:
df_regress = pd.read_csv('poker_dados_regress.csv')

In [3]:
df_regress.head(5)

,Raise,Position,Hand,Pot,P1-Action,P1-Bet,P2-Action,P2-Bet,P3-Action,P3-Bet,P4-Action,P4-Bet,P5-Action,P5-Bet
0,225.0,4,133,150.0,sb,50.0,bb,100.0,f,0.0,f,0.0,hold,0.0
1,300.0,0,5017,150.0,bb,100.0,f,0.0,f,0.0,f,0.0,f,0.0
2,200.0,5,441,150.0,sb,50.0,bb,100.0,f,0.0,f,0.0,f,0.0
3,200.0,4,1189,150.0,sb,50.0,bb,100.0,f,0.0,f,0.0,hold,0.0
4,500.0,5,705,350.0,sb,50.0,bb,100.0,f,0.0,f,0.0,r,200.0


In [4]:
colunas_numericas = ['Position',
                     'Hand',
                     'Pot',
                     'P1-Bet',
                     'P2-Bet',
                     'P3-Bet',
                     'P4-Bet',
                     'P5-Bet'
                    ]

colunas_classes = ['P1-Action',
                   'P2-Action',
                   'P3-Action',
                   'P4-Action',
                   'P5-Action'
                   ]

### Separando em Teste/Treino:

* **Separando o classificador:**

In [5]:
regress_X = df_regress.copy()
regress_X.drop('Raise', axis=1, inplace=True)
regress_y = df_regress['Raise']
regress_X.head(2)

,Position,Hand,Pot,P1-Action,P1-Bet,P2-Action,P2-Bet,P3-Action,P3-Bet,P4-Action,P4-Bet,P5-Action,P5-Bet
0,4,133,150.0,sb,50.0,bb,100.0,f,0.0,f,0.0,hold,0.0
1,0,5017,150.0,bb,100.0,f,0.0,f,0.0,f,0.0,f,0.0


<br>

* **Dividindo entre teste e treino :**

In [6]:
#Garantindo que cada coluna do teste e treino possua todas as classes para cada coluna
test_doesnt_have_balanced_values = True

while(test_doesnt_have_balanced_values):
    regress_X_train, regress_X_test, regress_y_train, regress_y_test = train_test_split(regress_X, regress_y, test_size=0.2)
    print(regress_X_train.shape)
    print(regress_X_test.shape)
    
    
    test_doesnt_have_balanced_values = False
    for coluna_classe in colunas_classes:
        if ( (set(df_regress[coluna_classe].unique()) != set(regress_X_train[coluna_classe].unique()) )
            or
              (set(df_regress[coluna_classe].unique()) != set(regress_X_test[coluna_classe].unique())  )
           ):
            
            test_doesnt_have_balanced_values = True
            print('-----')
            break

(6097, 13)
(1525, 13)


### Normalizando os dados:

* **Z-normalizando as colunas numéricas:**

In [7]:
regress_mean_train = regress_X_train[colunas_numericas].mean()
regress_std_train = regress_X_train[colunas_numericas].std(ddof=1)

regress_Z_train = regress_X_train.copy()
regress_Z_train[colunas_numericas] = regress_X_train[colunas_numericas] - regress_mean_train
regress_Z_train[colunas_numericas] = regress_Z_train[colunas_numericas] / regress_std_train

regress_Z_test = regress_X_test.copy()
regress_Z_test[colunas_numericas] = regress_X_test[colunas_numericas] - regress_mean_train
regress_Z_test[colunas_numericas] = regress_Z_test[colunas_numericas] / regress_std_train

* **Normalizando com OneHotEnconder as colunas com classes:**

In [8]:
regress_D_train = pd.get_dummies(regress_Z_train)
regress_D_test = pd.get_dummies(regress_Z_test)
regress_D_test.head(2)

,Position,Hand,Pot,P1-Bet,P2-Bet,P3-Bet,P4-Bet,P5-Bet,P1-Action_bb,P1-Action_c,...,P3-Action_hold,P3-Action_r,P4-Action_c,P4-Action_f,P4-Action_hold,P4-Action_r,P5-Action_c,P5-Action_f,P5-Action_hold,P5-Action_r
1151,0.068970,-0.906708,-0.402715,-0.164709,0.445854,-0.212559,-0.24947,-0.254638,0,0,...,0,0,0,0,1,0,0,0,1,0
7300,-0.520804,-0.468011,-0.402715,-0.164709,0.445854,-0.212559,-0.24947,-0.254638,0,0,...,1,0,0,0,1,0,0,0,1,0


## Modelo de regressão - kNN

### Validação Cruzada KNN:

In [9]:
regress_model_knn = KNeighborsRegressor()

#Gera a Grid
grid_search = GridSearchCV(regress_model_knn,
                           param_grid={"n_neighbors": range(1, 20)},
                           scoring="neg_mean_squared_error",
                           cv=7,
                           iid='deprecated')

#Realiza a busca com os valores do treino:
grid_search.fit(regress_D_train, regress_y_train)
best_n_neighbors = grid_search.best_estimator_.n_neighbors
print("Melhor número de vizinhos após substituir todos os valores na Grid: ", best_n_neighbors)

Melhor número de vizinhos após substituir todos os valores na Grid:  4


### Treino KNN:

In [10]:
regress_model_knn = KNeighborsRegressor(n_neighbors=best_n_neighbors)
regress_model_knn.fit(regress_D_train, regress_y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=4, p=2,
          weights='uniform')

### Teste KNN:

In [11]:
previsao_knn = regress_model_knn.predict(regress_D_test)

In [12]:
#Gerando dados da qualidade do modelo
knn_variancia_err = explained_variance_score(y_true=regress_y_test, y_pred=previsao_knn)
knn_r_quadrado_err = r2_score(y_true=regress_y_test, y_pred=previsao_knn)
knn_erro_medio_quadrado = mean_squared_error(y_true=regress_y_test, y_pred=previsao_knn)
knn_erro_medio_abs = median_absolute_error(y_true=regress_y_test, y_pred=previsao_knn)

score_knn = [knn_variancia_err, knn_r_quadrado_err, knn_erro_medio_abs, knn_erro_medio_quadrado]

<br>

## Modelo de regressão - Regressão Linear

### Validação Cruzada e Treino:

Não há hiper parâmetros para tunar utilizando regressão linear, mas para aprimorar o resultado forçaremos uma validação das features, gerando dois modelos lineares, um com validação cruzada e outro não: 

In [13]:
# Cria-se um esquema de validação cruzada para passar para o GridSeach
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# Range dos hiper parâmetros que serão tunados
hyper_params = [{'n_features_to_select': list(range(1, 29))}]

#Cria-se o modelo simples de RL
linear_model = LinearRegression()
linear_model.fit(regress_D_train, regress_y_train)
rfe = RFE(linear_model)             

# Cria-se um novo modelo com os novos pesos 
linear_model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = folds,
                        return_train_score=True,
                        iid='deprecated')      

linear_model_cv.fit(regress_D_train, regress_y_train) 

GridSearchCV(cv=KFold(n_splits=5, random_state=100, shuffle=True),
       error_score='raise-deprecating',
       estimator=RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
  n_features_to_select=None, step=1, verbose=0),
       fit_params=None, iid='deprecated', n_jobs=None,
       param_grid=[{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

### Teste:

In [14]:
previsao_lm = linear_model.predict(regress_D_test)
previsao_lm_cv = linear_model_cv.predict(regress_D_test)

In [15]:
#Gerando dados da qualidade do modelo linear
lm_variancia_err = explained_variance_score(y_true=regress_y_test, y_pred=previsao_lm)
lm_r_quadrado_err = r2_score(y_true=regress_y_test, y_pred=previsao_lm)
lm_erro_medio_quadrado = mean_squared_error(y_true=regress_y_test, y_pred=previsao_lm)
lm_erro_medio_abs = median_absolute_error(y_true=regress_y_test, y_pred=previsao_lm)

score_lm = [lm_variancia_err, lm_r_quadrado_err, lm_erro_medio_abs, lm_erro_medio_quadrado]

In [16]:
#Gerando dados da qualidade do modelo linear com validação cruzada
lm_cv_variancia_err = explained_variance_score(y_true=regress_y_test, y_pred=previsao_lm_cv)
lm_cv_r_quadrado_err = r2_score(y_true=regress_y_test, y_pred=previsao_lm_cv)
lm_cv_erro_medio_quadrado = mean_squared_error(y_true=regress_y_test, y_pred=previsao_lm_cv)
lm_cv_erro_medio_abs = median_absolute_error(y_true=regress_y_test, y_pred=previsao_lm_cv)

score_lm_cv = [lm_cv_variancia_err, lm_cv_r_quadrado_err, lm_cv_erro_medio_abs, lm_cv_erro_medio_quadrado]

# Comparando Modelos de Regressão:

In [17]:
comparacao = {}
comparacao['Regressão Linear'] = score_lm
comparacao['Regressão Linear c/ Validação Cruzada'] = score_lm_cv
comparacao['KNN'] = score_knn

df_comparacao = pd.DataFrame.from_dict(comparacao, orient='index', columns=['Variância', 'R^2', 'Erro Médio Absoluto', 'Erro Quadrado Médio'])
df_comparacao

,Variância,R^2,Erro Médio Absoluto,Erro Quadrado Médio
Regressão Linear,0.829650,0.829439,18.75,9138.855538
Regressão Linear c/ Validação Cruzada,0.826372,0.826242,18.00,9310.161311
KNN,0.843344,0.843333,16.75,8394.414959


O desempenho das regressões não foi muito bom, tendo valores de R2 próximos a 0,6. Isso pode ser possivelmente explicado pelo fato de que as apostas não podem ser muito facilmente previstas, devido à fatores aléatorios de decisão e a possíveis blefes, isto é, quando um jogador faz uma aposta com valor não condizente com sua força da mão ou outros atributos, por exemplo. As pontuações de erro, variância e R2 foram muito próximas para os três modelos, não havendo, provavelmente, diferença estatística relevante entre eles.